### Imports

In [1]:
# For Saving/Loading Files
from utils.storage import load_data,load_tokenizers

# -------------- Modelling Packages --------------
# For modeling
from keras.models import Model, Sequential
from keras.layers import Embedding, LSTM, Bidirectional
from keras.layers import Input, Reshape, SpatialDropout1D, Dense, Flatten
from keras.layers import Concatenate
from keras import optimizers

# Callback Functions
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

# For Timestamping Models
import time

# For balancing class weights
from sklearn.utils import class_weight

# -------------- General Packages --------------
# Data Manipulation
import numpy as np

# Saving/Loading
import os

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\kevin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-2e290b6ebd9c>", line 6, in <module>
    from keras.models import Model, Sequential
  File "C:\Users\kevin\Anaconda3\lib\site-packages\keras\__init__.py", line 3, in <module>
    from . import utils
  File "C:\Users\kevin\Anaconda3\lib\site-packages\keras\utils\__init__.py", line 6, in <module>
    from . import conv_utils
  File "C:\Users\kevin\Anaconda3\lib\site-packages\keras\utils\conv_utils.py", line 9, in <module>
    from .. import backend as K
  File "C:\Users\kevin\Anaconda3\lib\site-packages\keras\backend\__init__.py", line 1, in <module>
    from .load_backend import epsilon
  File "C:\Users\kevin\Anaconda3\lib\site-packages\keras\backend\load_backend.py", line 90, in <module>
    from .tensorflow_backend import *
  File "C:\Users\kevin\Anaconda3\lib\site-p

ModuleNotFoundError: No module named 'absl'

### Directories

In [2]:
MODEL_DIR = './models/'
SPLIT_DATA_DIR = './split_data/'
LOG_DIR = 'logs'
TOKENIZER_DIR = './tokenizers/'

### Loading the Dataset + Tokenizers

In [3]:
EN_X_train,EN_X_test,ZH_X_train,ZH_X_test,y_train,y_test = load_data(SPLIT_DATA_DIR)
t_EN,t_ZH = load_tokenizers(TOKENIZER_DIR)

### Creating the Models

In [4]:
os.makedirs(os.path.dirname(MODEL_DIR), exist_ok=True)

# -------------- Tokenizer Values --------------
EN_SENTENCE_SIZE = int(EN_X_train.shape[1]/2)
ZH_SENTENCE_SIZE = int(ZH_X_train.shape[1]/2)
en_vocab_size = 35000
zh_vocab_size = 70000
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

# -------------- TUNABLE HYPERPARAMETERS --------------
EMBED_SIZES = [ 50 ]
LSTM_SIZES = [ 100 ]
LSTM_LAYER_SIZES = [ 1 ]
DROPOUT_SIZES = [ (0.75,0.25) ]
# DROPOUT_SIZES = [(0,0),(0.5,0),(0,0.5),(0.5,0.5),
#                  (0.25,0.25),(0.25,0.5),(0.5,0.25),(0.25,0),
#                  (0,0.25),(0.75,0.75),(0.75,0.5),(0.5,0.75),
#                  (0.75,0),(0,0.75),(0.75,0.25),(0.25,0.75)]

loss = 'sparse_categorical_crossentropy'
optimizer = optimizers.adam()
#optimizer = optimizers.sgd(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
metrics = ['accuracy']

epochs = 200
batch_size = 1024
PATIENCE = 3

In [5]:
for EMBED_SIZE in EMBED_SIZES:
    for DROPOUT_SIZE in DROPOUT_SIZES:
        for LSTM_SIZE in LSTM_SIZES:
            for LSTM_LAYERS in LSTM_LAYER_SIZES:
                
                # -------------- MODEL NAMING --------------
                NAME = 'BiLSTM-{}E-{}x{}L-{}Dropout-{}.hdf5'.format(EMBED_SIZE,
                                                                         LSTM_LAYERS,LSTM_SIZE,
                                                                         DROPOUT_SIZE,
                                                                         time.time())
                print('Creating {}'.format(NAME))
                MODEL_LOG_DIR = os.path.join(LOG_DIR,NAME)

                # -------------- Callbacks --------------
                # access tensorboard from the command line: tensorboard --logdir=logs/
                tensorboard = TensorBoard(log_dir=MODEL_LOG_DIR) 
                checkpointer = ModelCheckpoint(MODEL_DIR+NAME, 
                                               monitor='val_accuracy', 
                                               verbose=1, 
                                               save_best_only=True, 
                                               mode='auto')
                earlystop = EarlyStopping(monitor='val_loss', patience=PATIENCE)

                callbacks=[tensorboard,checkpointer,earlystop]

                # -------------- EN MODEL CREATION --------------
                EN_INPUT = Input(shape=(EN_SENTENCE_SIZE*2,))
                EN_MODEL = Reshape((-1,2,EN_SENTENCE_SIZE))(EN_INPUT)
                EN_MODEL = Embedding(en_vocab_size,
                                EMBED_SIZE,
                                input_shape=(2,EN_SENTENCE_SIZE),
                                trainable=True)(EN_MODEL)
                EN_MODEL = Reshape((2,EN_SENTENCE_SIZE*EMBED_SIZE,))(EN_MODEL)
                
                if DROPOUT_SIZE[0] > 0: EN_MODEL = SpatialDropout1D(DROPOUT_SIZE[0])(EN_MODEL)
                for layer in range(LSTM_LAYERS-1):
                    EN_MODEL = Bidirectional(LSTM(LSTM_SIZE,
                                                  return_sequences=True, 
                                                  recurrent_dropout=DROPOUT_SIZE[1]))(EN_MODEL)
                EN_MODEL = Bidirectional(LSTM(LSTM_SIZE,
                                              return_sequences=True,
                                              recurrent_dropout=DROPOUT_SIZE[1]))(EN_MODEL)
    
                # -------------- ZH MODEL CREATION --------------
                ZH_INPUT = Input(shape=(ZH_SENTENCE_SIZE*2,))
                ZH_MODEL = Reshape((-1,2,ZH_SENTENCE_SIZE))(ZH_INPUT)
                ZH_MODEL = Embedding(zh_vocab_size,
                                EMBED_SIZE,
                                input_shape=(2,ZH_SENTENCE_SIZE),
                                trainable=True)(ZH_MODEL)
                ZH_MODEL = Reshape((2,ZH_SENTENCE_SIZE*EMBED_SIZE,))(ZH_MODEL)
                
                if DROPOUT_SIZE[0] > 0: ZH_MODEL = SpatialDropout1D(DROPOUT_SIZE[0])(ZH_MODEL)
                for layer in range(LSTM_LAYERS-1):
                    ZH_MODEL = Bidirectional(LSTM(LSTM_SIZE,
                                                  return_sequences=True, 
                                                  recurrent_dropout=DROPOUT_SIZE[1]))(ZH_MODEL)
                ZH_MODEL = Bidirectional(LSTM(LSTM_SIZE,
                                              return_sequences=True,
                                              recurrent_dropout=DROPOUT_SIZE[1]))(ZH_MODEL)
                    
                # -------------- MERGE MODEL --------------
                merged = Concatenate(1)([EN_MODEL,ZH_MODEL])
                merged = LSTM(LSTM_SIZE,
                              return_sequences=True,
                              recurrent_dropout=DROPOUT_SIZE[1])(merged)
                merged = Flatten()(merged)
                merged = Dense(3, activation='softmax')(merged)

                model = Model(inputs=[EN_INPUT,ZH_INPUT], outputs=merged)
                model.compile(optimizer=optimizer, loss=loss,metrics=metrics)

                # -------------- Training the model --------------
                model.fit([EN_X_train,ZH_X_train], y_train,
                          epochs=epochs,
                          batch_size=batch_size,
                          validation_data=([EN_X_test,ZH_X_test], y_test),
                          callbacks=callbacks,
                          class_weight=class_weights)

Creating BiLSTM-50E-1x100L-(0.75, 0.25)Dropout-1574663940.6780815.hdf5


C:\Users\kevin\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 192223 samples, validate on 64087 samples
Epoch 1/200
192223/192223 [==============================] - 58s 303us/step - loss: 0.6005 - accuracy: 0.7369 - val_loss: 0.4210 - val_accuracy: 0.8142

Epoch 00001: val_accuracy improved from -inf to 0.81421, saving model to ./models/BiLSTM-50E-1x100L-(0.75, 0.25)Dropout-1574663940.6780815.hdf5
Epoch 2/200
192223/192223 [==============================] - 51s 267us/step - loss: 0.3919 - accuracy: 0.8266 - val_loss: 0.3713 - val_accuracy: 0.8382

Epoch 00002: val_accuracy improved from 0.81421 to 0.83824, saving model to ./models/BiLSTM-50E-1x100L-(0.75, 0.25)Dropout-1574663940.6780815.hdf5
Epoch 3/200
192223/192223 [==============================] - 51s 266us/step - loss: 0.3324 - accuracy: 0.8567 - val_loss: 0.3557 - val_accuracy: 0.8495

Epoch 00003: val_accuracy improved from 0.83824 to 0.84952, saving model to ./models/BiLSTM-50E-1x100L-(0.75, 0.25)Dropout-1574663940.6780815.hdf5
Epoch 4/200
192223/192223 [=========================